In [1]:
%pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate ftfy spacy

     |████████████████████████████████| 503 kB 15.1 MB/s 
     |████████████████████████████████| 5.8 MB 36.8 MB/s 
     |████████████████████████████████| 33.8 MB 285 kB/s 
     |████████████████████████████████| 191 kB 60.2 MB/s 
     |████████████████████████████████| 53 kB 892 kB/s 
     |████████████████████████████████| 6.6 MB 42.7 MB/s 
     |████████████████████████████████| 182 kB 65.3 MB/s 
     |████████████████████████████████| 7.6 MB 47.5 MB/s 
     |████████████████████████████████| 1.6 MB 57.1 MB/s 


In [2]:
import subprocess

# The xformers package is mandatory to be able to create several 768x768 images.
github_url = "https://github.com/brian6091/xformers-wheels"
xformer_id = "0.0.15.dev0+4c06c79"
xformers_wheels = f"xformers-{xformer_id}.d20221205-cp38-cp38-linux_x86_64.whl"

# Install xformers using pre-compiled Python wheels
%pip install -q {github_url}/releases/download/{xformer_id}/{xformers_wheels}

     |████████████████████████████████| 47.4 MB 66 kB/s 


In [3]:
model_id = "stabilityai/stable-diffusion-2-1"

In [4]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, DPMSolverMultistepScheduler
scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [5]:
import mediapy as media
import torch
from diffusers import StableDiffusionPipeline

device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    torch_dtype=torch.float16,
    revision="fp16",
    )
pipe = pipe.to(device)

pipe.enable_xformers_memory_efficient_attention()

if model_id.endswith('-base'):
  image_length = 512
else:
  image_length = 768

Downloading:   0%|          | 0.00/517 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/351 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/628 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/999 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

In [10]:
from torch import autocast
prompt = "Cinematic cyberpunk artificial intelligence"
num_images = 2

generator = torch.Generator("cuda").manual_seed(1)

with autocast("cuda"):
    images = pipe(
        prompt,
        num_images_per_prompt=num_images,
        guidance_scale=9,
        num_inference_steps=50,
        height=image_length,
        width=image_length,
        generator=generator
        ).images
    
media.show_images(images)

Output hidden; open in https://colab.research.google.com to view.